# 1. Inserir os eventos no arquivo
# 2. Análise espectral
# 3. Filtrar os dados em bandas



In [ ]:
# Carregando pacotes
from funcoes_uteis import sep_horas, time_to_sec, convert_time_sec, criar_eventos
from funcoes_uteis import selecionar_intervalos, selecionar_tentativas
import numpy as np
import mne
from mne.time_frequency import psd_array_multitaper
import pandas as pd
import matplotlib

In [ ]:
# Definir o número de tentativas
n_tt = 20

#Truque para "plotar" os gráficos em uma janela separada
matplotlib.use('Qt5Agg')

In [ ]:
#Abrindo os dados pré-processados

nome_arquivo = 'eeg/preproc/1/ME.fif'
raw = mne.io.read_raw_fif(nome_arquivo, preload=True)
raw.plot()

#Inserindo o sistema 10-20
easycap_montage = mne.channels.make_standard_montage("easycap-M1")
raw.set_montage(easycap_montage)

In [ ]:
#Abrindo os dados comportamentais
nome_arquivo = 'comp/1/ME'
dados_comp = pd.read_csv(nome_arquivo,sep='\t',header=None)


In [ ]:

#Pegando a hora que iniciou a gravação do eeg e transformando em segudos

h, m,s = raw.info['meas_date'].hour, raw.info['meas_date'].minute, raw.info['meas_date'].second
T0 = h * 3600 + m * 60 + s
print(T0)

In [ ]:
#Converter o tempo para segundos
# 1. quais colunas do arquivo comportamental os eventos? 

ref_col1 = 14
ref_col2 = 19

time_sec_menosT0 = convert_time_sec (dados_comp,n_tt,T0,ref_col1,ref_col2)
print(time_sec_menosT0)
    

In [ ]:
# Criando um novo canal para os eventos
### CUIDADO PARA NÃO "RODAR" ESSA CÉLULA DUAS VEZES
criar_eventos(raw,time_sec_menosT0, n_tt) #Sempre será com uma tentativa a menos, no caso a primeira
raw.plot() 


In [ ]:
# Gerar o espectro - 4 a 40 Hz é uma boa alterativa
espectro  = raw.compute_psd(method='multitaper', fmin=4, fmax=40)
print(espectro)

In [ ]:
# Três formas de representar a análise espectral
# Primeira
espectro.plot(picks="data", exclude="bads")




In [ ]:
# Segunda
espectro.plot_topo(color="k", fig_facecolor="w", axis_facecolor="w")


In [ ]:
# Terceira
bandas_escolhidas = {'Theta (4-8 Hz)': (4, 8),
         'Alpha (8-12 Hz)': (8, 12), 'Beta (12-30 Hz)': (12, 30),
         'Gamma (30-45 Hz)': (30, 45)}

espectro.plot_topomap(bands=bandas_escolhidas, agg_fun=np.median)


In [ ]:
# Salvar o psd 

psd, freq = psd_array_multitaper(raw.get_data(), sfreq=raw.info['sfreq'], fmin=4,fmax=45)


# 3 primeira freq dos 4 primeiros canais:
print(psd[:4, :3])
print(freq [:4])



In [ ]:
# Selecionar fequencias para salvar
banda = [8,12]


#### selecinando os indices
for i in range (len(freq)):
    indice_inicial = 0
    
    if freq[i] >= banda[0]:
        indice_inicial = i
        break
for i in range (len(freq)):
    indice_final = 0
    
    if freq[i] >= banda[1]:
        indice_final = i
        break   
    
   
print(freq[indice_inicial:indice_final]) #conferindo as frequencias

In [ ]:
# Salvar os dados na planilha

np.savetxt('eeg/espectro/suj1.csv', psd[:,indice_inicial:indice_final], delimiter=',')

print('linhas:', raw.ch_names) #para lembrar
#print('colunas:', freq[indice_inicial:indice_final])

In [ ]:
############# Filtrar os dados em bandas##########

lowcut, highcut = 10,12
raw_fill = raw.copy().filter(l_freq=lowcut, h_freq=highcut)


raw_fill.plot()


In [ ]:
# Se quiser salvar os dados
np.savetxt('eeg/filtrado/suj1.csv', raw_fill.get_data(), delimiter=',')

print('linhas:', raw_fill.ch_names) #para lembrar

In [ ]:
# Separar os dados por tentativas

eeg_interesse = raw_fill.copy() # Dados filtrados

inicio, fim = time_sec_menosT0[16] #0 = tentativa 1, ...


raw_cortado = eeg_interesse.crop(tmin=inicio, tmax=fim)
print('duracao', str(fim-inicio))

In [ ]:
#Plotar a tentativa
raw_cortado.plot()


In [ ]:
# Salvar a tentivas recortada
raw_cortado.save('tentativa1.fif', overwrite=True)